# PREPROCESSING ON THE SAMPLED_DATASET

In [1]:
import pandas as pd
df = pd.read_csv("sampled_dataset.csv")
df.head()

,title,artist,year,views,features,lyrics,is_country,is_pop,is_rap,is_rb,is_rock,tag
0,Something in the Water,Pokey LaFarge,2015,10902,{},"She got a broke down El Camino, in the front y...",True,False,False,False,False,is_country
1,The Last Event,Matthew Ryan,2019,34,{},Later on\nWhen we've all come down\nAnd the st...,True,False,False,False,False,is_country
2,Friends With The Enemy,Poor Man's Poison,2011,5611,"{""Poor Man\\'s Poison""}",[Verse 1]\nLonging for something more\nWill I ...,True,False,False,False,False,is_country
3,Sleepin With the Radio On,Charly McClain,1981,100,{},My heart beats with the lonely rain\nWishin' I...,True,False,False,False,False,is_country
4,Riders in the Sky,Michael Martin Murphey,1993,108,{},An old cowpoke went ridin' out one dark and wi...,True,False,False,False,False,is_country


### PULIZIA LYRICS

In [5]:
#ripulire lyrics tenendo solo tag importanti per divisione in strofe

import re
keep_pattern = r"\[(Chorus|Verse|Bridge|Intro|Outro|Hook|Prehook|Posthook|Introduction|Interlude|Coda|Conclusion|Refrain).\]*"

# Function to remove undesired bracketed text
def clean_lyrics(lyrics):
    # Remove any text inside brackets that isn't a recognized section
    cleaned_lyrics = re.sub(r"\[.*?\]", lambda match: match.group(0) if re.match(keep_pattern, match.group(0)) else "", lyrics)
    return cleaned_lyrics

# Example usage with a DataFrame
df['cleaned_lyrics'] = df['lyrics'].apply(clean_lyrics)

In [6]:
#function to split the stanzas with different criterion   

def process_lyric(ly):
    sections = [ly.strip()]
    
    # First split by the pattern
    pattern = r"\[(Introduction|Interlude|Coda|Hook|Prehook|Posthook|Conclusion|Refrain|Verse|Intro|Outro|Chorus|Bridge).\]*"
    if re.search(pattern, ly, re.IGNORECASE):
        sections = [segment for section in sections for segment in section.split("\n[")]

    # Further split by "Verse"
    if re.search(r"Verse", ly, re.IGNORECASE):
        sections = [segment for section in sections for segment in section.split("Verse")]

    # Further split by double newlines
    if "\n\n" in ly:
        sections = [segment for section in sections for segment in section.split("\n\n")]
    
    # Further split by "(Chorus)"
    if re.search(r"(Chorus)", ly, re.IGNORECASE):
        sections = [segment for section in sections for segment in section.split("(")]

    # Clean up whitespace for each section and remove empty sections
    return [section.strip() for section in sections if section.strip()]

# Apply the function to the cleaned_lyrics column
df["processed_lyrics"] = df["cleaned_lyrics"].apply(process_lyric)

In [7]:
#function to check if nella lista di stringhe ci sono delle stringhe con solo "hook", "chorus" ecc. e stringhe vuote e in caso eliminarle

def clean_list(lyrics_list):
    if not isinstance(lyrics_list, list):  # Ensure the input is a list
        return lyrics_list
    
    # Regex pattern to match tags like "Hook]", "Chorus]", etc.
    tag_pattern = r"^\s*(Hook|Chorus|Bridge|Verse|Outro|Intro|Refrain|Prehook|Posthook|Coda|Interlude|Conclusion)\s*.*?\]\s*$"
    #tag_pattern = r"^\s*(Hook|Chorus|Bridge|Verse|Outro|Intro|Refrain|Prehook|Posthook|Coda|Interlude|Conclusion)\s*.*?\]?\s*$" questo mette la parentesi in fondo opzionale
    
    # Filter out empty strings and strings matching the tag pattern
    return [line for line in lyrics_list if line and not re.match(tag_pattern, line, re.IGNORECASE)]


# Apply the cleaning function to the specified column
df['cleaned_lists'] = df["processed_lyrics"].apply(clean_list)

In [8]:
df.head()

,title,artist,year,views,features,lyrics,is_country,is_pop,is_rap,is_rb,is_rock,tag,cleaned_lyrics,processed_lyrics,cleaned_lists
0,Something in the Water,Pokey LaFarge,2015,10902,{},"She got a broke down El Camino, in the front y...",True,False,False,False,False,is_country,"She got a broke down El Camino, in the front y...","[She got a broke down El Camino, in the front ...","[She got a broke down El Camino, in the front ..."
1,The Last Event,Matthew Ryan,2019,34,{},Later on\nWhen we've all come down\nAnd the st...,True,False,False,False,False,is_country,Later on\nWhen we've all come down\nAnd the st...,[Later on\nWhen we've all come down\nAnd the s...,[Later on\nWhen we've all come down\nAnd the s...
2,Friends With The Enemy,Poor Man's Poison,2011,5611,"{""Poor Man\\'s Poison""}",[Verse 1]\nLonging for something more\nWill I ...,True,False,False,False,False,is_country,[Verse 1]\nLonging for something more\nWill I ...,"[[, 1]\nLonging for something more\nWill I eve...","[[, 1]\nLonging for something more\nWill I eve..."
3,Sleepin With the Radio On,Charly McClain,1981,100,{},My heart beats with the lonely rain\nWishin' I...,True,False,False,False,False,is_country,My heart beats with the lonely rain\nWishin' I...,[My heart beats with the lonely rain\nWishin' ...,[My heart beats with the lonely rain\nWishin' ...
4,Riders in the Sky,Michael Martin Murphey,1993,108,{},An old cowpoke went ridin' out one dark and wi...,True,False,False,False,False,is_country,An old cowpoke went ridin' out one dark and wi...,[An old cowpoke went ridin' out one dark and w...,[An old cowpoke went ridin' out one dark and w...


### CONTROLLO SU PULIZIA LYRICS

- raffinare regex
- controllare se non stiamo togliendo troppo
- fare explode

In [12]:
x = df.loc[2, "processed_lyrics"]
x

['[',
 '1]\nLonging for something more\nWill I ever be free?\nWill I ever be free?\nI been waitin’ for that sunshine on my door\nWill I ever be free?\nOh Lord set me free\nYou know you’ll never be free\nWhen you’re friends with the enemy',
 '2]\nWorkin’ on a better way\nWill I ever be free?\nWill I ever be free?\nKeeping on even though it’s oh so far away\nWill I ever be free?\nOh Lord set me free\nYou know you’ll never be free\nWhen you’re friends with the enemy',
 'Chorus]\nAnd them hard times keep comin’ at the worst times\nAnd it’s hard enough just to keep my head afloat\nAnd I pray to God that I make it through December\nI’ll be on my way now walking this stretch alone\nSaid I’ll be on my way, I’m walking this stretch alone',
 'Interlude]',
 '3]\nNever more will you see my face\nWill I ever be free?\nWill I ever be free?\nBroken down but I’m holding out for better days\nWill I ever be free?\nOh Lord set me free\nYou know you’ll never be free\nWhen you’re friends with the enemy',
 

In [13]:
x = df.loc[2, "cleaned_lists"]
x

['[',
 '1]\nLonging for something more\nWill I ever be free?\nWill I ever be free?\nI been waitin’ for that sunshine on my door\nWill I ever be free?\nOh Lord set me free\nYou know you’ll never be free\nWhen you’re friends with the enemy',
 '2]\nWorkin’ on a better way\nWill I ever be free?\nWill I ever be free?\nKeeping on even though it’s oh so far away\nWill I ever be free?\nOh Lord set me free\nYou know you’ll never be free\nWhen you’re friends with the enemy',
 'Chorus]\nAnd them hard times keep comin’ at the worst times\nAnd it’s hard enough just to keep my head afloat\nAnd I pray to God that I make it through December\nI’ll be on my way now walking this stretch alone\nSaid I’ll be on my way, I’m walking this stretch alone',
 '3]\nNever more will you see my face\nWill I ever be free?\nWill I ever be free?\nBroken down but I’m holding out for better days\nWill I ever be free?\nOh Lord set me free\nYou know you’ll never be free\nWhen you’re friends with the enemy',
 'Chorus]\nAnd t

### EXPLODE SAMPLED_DATA

In [ ]:
#removing redundant variables
df = df.drop(['cleaned_lyrics', 'processed_lyrics'], axis=1)

In [ ]:
exploded_df = df.explode('cleaned_lists', ignore_index=False)
exploded_df.rename(columns={'cleaned_lists': 'stanzas'}, inplace=True)

exploded_df